In [1]:
import re
import string
import operator
import functools
import glob
import os
import matplotlib.colors as mc
import nltk
import numpy as np
import textstat
import unicodedata
import pandas as pd

import textstat
from lexicalrichness import LexicalRichness
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag, map_tag
from nltk.tokenize import sent_tokenize

 # Feature Selection
from sklearn.feature_selection import VarianceThreshold


from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

In [7]:
engine = create_engine("sqlite:///mentoriam11.sqlite3", echo=True)


# Crea una clase base declarativa
Base = declarative_base()

# Define la clase "Documento"
class Documento(Base):
    __tablename__ = 'documento'

    id_documento = Column(Integer, primary_key=True)
    filename = Column(String)
    text = Column(String)
    segmentos = relationship('Segmento', backref='documento')

# Define la clase "Segmento"
class Segmento(Base):
    __tablename__ = 'segmento'

    id_segmento = Column(Integer, primary_key=True)
    segmento_texto = Column(String)
    segmento_limpio = Column(String)
    init_s = Column(Integer)
    length = Column(Integer)
    id_documento = Column(Integer, ForeignKey('documento.id_documento'))

# Crea las tablas en la base de datos
Base.metadata.create_all(engine)

2023-07-12 15:43:55,038 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-12 15:43:55,039 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("documento")
2023-07-12 15:43:55,039 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-07-12 15:43:55,040 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("documento")
2023-07-12 15:43:55,040 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-07-12 15:43:55,040 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("segmento")
2023-07-12 15:43:55,041 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-07-12 15:43:55,041 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("segmento")
2023-07-12 15:43:55,042 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-07-12 15:43:55,042 INFO sqlalchemy.engine.Engine 
CREATE TABLE documento (
	id_documento INTEGER NOT NULL, 
	filename VARCHAR, 
	text VARCHAR, 
	PRIMARY KEY (id_documento)
)


2023-07-12 15:43:55,043 INFO sqlalchemy.engine.Engine [no key 0.00032s] ()
2023-07-12 15:43:55,045 INFO sqlalchemy.engine.

C:\Users\pablonicolasr\AppData\Local\Temp\ipykernel_3628\2669130117.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [6]:
df = pd.read_csv(os.path.join(os.getcwd(), "intrinsic.csv"))
df

,index,filename,text
0,0,suspicious-document00001.txt,The convening of this body gave rise to much d...
1,1,suspicious-document00002.txt,WOMAN\n\nMAN'S EQUAL.\n\nBY\n\nRev. THOS. WEBS...
2,2,suspicious-document00003.txt,"I do not remember that I was ""very angry"" abou..."
3,3,suspicious-document00004.txt,"Tulaji Angria was dark, inclined to be fat, ..."
4,4,suspicious-document00005.txt,Hiroshima was the primary target of the first ...
...,...,...,...
4747,4747,suspicious-document04748.txt,STORIES THAT\n\n WO...
4748,4748,suspicious-document04749.txt,"The legendary history of the place, both tempo..."
4749,4749,suspicious-document04750.txt,"The detective lunged toward her. ""Just what I ..."
4750,4750,suspicious-document04751.txt,Redactor's note: Italics are indicated by unde...
